<a href="https://colab.research.google.com/github/SYED-ABDULRAHMAN/Project/blob/main/movie_recomendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Data Citation:
# F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. ACM Transactions on
# Interactive Intelligent Systems (TiiS) 5, 4: 19:1–19:19.

! curl http://files.grouplens.org/datasets/movielens/ml-latest-small.zip -o ml-latest-small.zip


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  955k  100  955k    0     0  1034k      0 --:--:-- --:--:-- --:--:-- 1033k


In [ ]:
import zipfile
with zipfile.ZipFile('ml-latest-small.zip', 'r') as zip_ref:
    zip_ref.extractall('data')


In [ ]:
# import the dataset
import pandas as pd
movies_df = pd.read_csv('data/ml-latest-small/movies.csv')
ratings_df = pd.read_csv('data/ml-latest-small/ratings.csv')

In [ ]:
print('The dimensions of movies dataframe are:', movies_df.shape,'\nThe dimensions of ratings dataframe are:', ratings_df.shape)


The dimensions of movies dataframe are: (9742, 3) 
The dimensions of ratings dataframe are: (100836, 4)


In [ ]:
# Take a look at movies_df
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
# Take a look at ratings_df
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [ ]:
# Movie ID to movie name mapping
movie_names = movies_df.set_index('movieId')['title'].to_dict()
n_users = len(ratings_df.userId.unique())
n_items = len(ratings_df.movieId.unique())
print("Number of unique users:", n_users)
print("Number of unique movies:", n_items)
print("The full rating matrix will have:", n_users*n_items, 'elements.')
print('----------')
print("Number of ratings:", len(ratings_df))
print("Therefore: ", len(ratings_df) / (n_users*n_items) * 100, '% of the matrix is filled.')
print("We have an incredibly sparse matrix to work with here.")
print("And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2")
print("You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.")
print("One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data")


Number of unique users: 610
Number of unique movies: 9724
The full rating matrix will have: 5931640 elements.
----------
Number of ratings: 100836
Therefore:  1.6999683055613624 % of the matrix is filled.
We have an incredibly sparse matrix to work with here.
And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2
You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.
One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data


In [ ]:
import torch
import numpy as np
from torch.autograd import Variable
from tqdm import tqdm_notebook as tqdm

class MatrixFactorization(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        # create user embeddings
        self.user_factors = torch.nn.Embedding(n_users, n_factors) # think of this as a lookup table for the input.
        # create item embeddings
        self.item_factors = torch.nn.Embedding(n_items, n_factors) # think of this as a lookup table for the input.
        self.user_factors.weight.data.uniform_(0, 0.05)
        self.item_factors.weight.data.uniform_(0, 0.05)

    def forward(self, data):
        # matrix multiplication
        users, items = data[:,0], data[:,1]
        return (self.user_factors(users)*self.item_factors(items)).sum(1)
    # def forward(self, user, item):
    # 	# matrix multiplication
    #     return (self.user_factors(user)*self.item_factors(item)).sum(1)

    def predict(self, user, item):
        return self.forward(user, item)


In [ ]:
# Creating the dataloader (necessary for PyTorch)
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader # package that helps transform your data to machine learning readiness

# Note: This isn't 'good' practice, in a MLops sense but we'll roll with this since the data is already loaded in memory.
class Loader(Dataset):
    def __init__(self):
        self.ratings = ratings_df.copy()

        # Extract all user IDs and movie IDs
        users = ratings_df.userId.unique()
        movies = ratings_df.movieId.unique()

        #--- Producing new continuous IDs for users and movies ---

        # Unique values : index
        self.userid2idx = {o:i for i,o in enumerate(users)}
        self.movieid2idx = {o:i for i,o in enumerate(movies)}

        # Obtained continuous ID for users and movies
        self.idx2userid = {i:o for o,i in self.userid2idx.items()}
        self.idx2movieid = {i:o for o,i in self.movieid2idx.items()}

        # return the id from the indexed values as noted in the lambda function down below.
        self.ratings.movieId = ratings_df.movieId.apply(lambda x: self.movieid2idx[x])
        self.ratings.userId = ratings_df.userId.apply(lambda x: self.userid2idx[x])


        self.x = self.ratings.drop(['rating', 'timestamp'], axis=1).values
        self.y = self.ratings['rating'].values
        self.x, self.y = torch.tensor(self.x), torch.tensor(self.y) # Transforms the data to tensors (ready for torch models.)

    def __getitem__(self, index):
        return (self.x[index], self.y[index])

    def __len__(self):
        return len(self.ratings)


In [ ]:
num_epochs = 128
cuda = torch.cuda.is_available()

print("Is running on GPU:", cuda)

model = MatrixFactorization(n_users, n_items, n_factors=8)
print(model)
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
# GPU enable if you have a GPU...
if cuda:
    model = model.cuda()

# MSE loss
loss_fn = torch.nn.MSELoss()

# ADAM optimizier
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Train data
train_set = Loader()
train_loader = DataLoader(train_set, 128, shuffle=True)

Is running on GPU: True
MatrixFactorization(
  (user_factors): Embedding(610, 8)
  (item_factors): Embedding(9724, 8)
)
user_factors.weight tensor([[0.0446, 0.0278, 0.0072,  ..., 0.0083, 0.0134, 0.0408],
        [0.0277, 0.0485, 0.0384,  ..., 0.0150, 0.0334, 0.0028],
        [0.0467, 0.0148, 0.0158,  ..., 0.0365, 0.0469, 0.0034],
        ...,
        [0.0142, 0.0430, 0.0351,  ..., 0.0305, 0.0128, 0.0027],
        [0.0439, 0.0386, 0.0159,  ..., 0.0424, 0.0058, 0.0265],
        [0.0313, 0.0230, 0.0251,  ..., 0.0314, 0.0266, 0.0033]])
item_factors.weight tensor([[0.0346, 0.0217, 0.0381,  ..., 0.0496, 0.0006, 0.0244],
        [0.0044, 0.0082, 0.0013,  ..., 0.0185, 0.0053, 0.0003],
        [0.0148, 0.0470, 0.0429,  ..., 0.0416, 0.0438, 0.0298],
        ...,
        [0.0392, 0.0103, 0.0306,  ..., 0.0285, 0.0415, 0.0281],
        [0.0102, 0.0403, 0.0083,  ..., 0.0068, 0.0139, 0.0465],
        [0.0364, 0.0143, 0.0022,  ..., 0.0158, 0.0138, 0.0174]])


In [ ]:
for it in tqdm(range(num_epochs)):
    losses = []
    for x, y in train_loader:
         if cuda:
            x, y = x.cuda(), y.cuda()
            optimizer.zero_grad()
            outputs = model(x)
            loss = loss_fn(outputs.squeeze(), y.type(torch.float32))
            losses.append(loss.item())
            loss.backward()
            optimizer.step()
    print("iter #{}".format(it), "Loss:", sum(losses) / len(losses))

<ipython-input-44-dad152416852>:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for it in tqdm(range(num_epochs)):


  0%|          | 0/128 [00:00<?, ?it/s]

iter #0 Loss: 11.064781530254383
iter #1 Loss: 4.748451604758422
iter #2 Loss: 2.476534513803908
iter #3 Loss: 1.7212999469888997
iter #4 Loss: 1.345843861550849
iter #5 Loss: 1.1286833738615065
iter #6 Loss: 0.991398822535113
iter #7 Loss: 0.9003690971335784
iter #8 Loss: 0.8375775329201355
iter #9 Loss: 0.7921489159846063
iter #10 Loss: 0.7593920519811853
iter #11 Loss: 0.735051458824407
iter #12 Loss: 0.7160773361621774
iter #13 Loss: 0.7017755796234619
iter #14 Loss: 0.6904297062588222
iter #15 Loss: 0.6817847135557136
iter #16 Loss: 0.6751219834470507
iter #17 Loss: 0.6697932145922317
iter #18 Loss: 0.6655280330610759
iter #19 Loss: 0.6627348208336661
iter #20 Loss: 0.6603349826965236
iter #21 Loss: 0.6593323414247048
iter #22 Loss: 0.6577473089930975
iter #23 Loss: 0.656815837217769
iter #24 Loss: 0.6559686032572979
iter #25 Loss: 0.6551083409922377
iter #26 Loss: 0.6543474567647513
iter #27 Loss: 0.6534151378546269
iter #28 Loss: 0.6527854438663134
iter #29 Loss: 0.6509534929912

In [ ]:
# By training the model, we will have tuned latent factors for movies and users.
c = 0
uw = 0
iw = 0
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
        if c == 0:
          uw = param.data
          c +=1
        else:
          iw = param.data
        #print('param_data', param_data)


user_factors.weight tensor([[ 1.6724,  0.7930,  1.1878,  ...,  1.6194,  1.3485,  0.9581],
        [ 1.5207,  0.5290,  1.1725,  ...,  0.8079,  0.6990,  1.9342],
        [-2.0148,  1.2770, -1.1820,  ...,  2.7658,  0.5197,  0.0075],
        ...,
        [-0.0076,  2.1945,  0.9481,  ...,  1.9501,  0.4848,  0.1561],
        [ 0.9848,  1.1655,  1.4740,  ...,  0.6311,  0.4394,  1.0589],
        [ 0.0987,  1.1129,  1.1059,  ...,  0.8338,  1.6565,  1.0576]],
       device='cuda:0')
item_factors.weight tensor([[0.3884, 0.2936, 0.5835,  ..., 0.4037, 0.5849, 0.8155],
        [0.6958, 0.1525, 0.8152,  ..., 0.4944, 0.3219, 0.3641],
        [0.0444, 0.4879, 0.6322,  ..., 0.4295, 0.5846, 0.5450],
        ...,
        [0.3213, 0.3196, 0.3390,  ..., 0.3382, 0.3522, 0.3370],
        [0.3243, 0.4231, 0.3910,  ..., 0.3907, 0.3993, 0.4299],
        [0.3891, 0.3935, 0.3805,  ..., 0.3961, 0.3922, 0.3949]],
       device='cuda:0')


In [ ]:
trained_movie_embeddings = model.item_factors.weight.data.cpu().numpy()

In [ ]:
len(trained_movie_embeddings) # unique movie factor weights


9724

In [ ]:
from sklearn.cluster import KMeans
# Fit the clusters based on the movie weights
kmeans = KMeans(n_clusters=10, random_state=0).fit(trained_movie_embeddings)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
'''It can be seen here that the movies that are in the same cluster tend to have
similar genres. Also note that the algorithm is unfamiliar with the movie name
and only obtained the relationships by looking at the numbers representing how
users have responded to the movie selections.'''
for cluster in range(10):
  print("Cluster #{}".format(cluster))
  movs = []
  for movidx in np.where(kmeans.labels_ == cluster)[0]:
    movid = train_set.idx2movieid[movidx]
    rat_count = ratings_df.loc[ratings_df['movieId']==movid].count()[0]
    movs.append((movie_names[movid], rat_count))
  for mov in sorted(movs, key=lambda tup: tup[1], reverse=True)[:10]:
    print("\t", mov[0])

Cluster #0
	 Pulp Fiction (1994)
	 Fight Club (1999)
	 Usual Suspects, The (1995)
	 American Beauty (1999)
	 Seven (a.k.a. Se7en) (1995)
	 Lord of the Rings: The Fellowship of the Ring, The (2001)
	 Godfather, The (1972)
	 Lord of the Rings: The Two Towers, The (2002)
	 Lord of the Rings: The Return of the King, The (2003)
	 Fargo (1996)
Cluster #1
	 Toy Story (1995)
	 Apollo 13 (1995)
	 Fugitive, The (1993)
	 Batman (1989)
	 Aladdin (1992)
	 Lion King, The (1994)
	 Speed (1994)
	 Shrek (2001)
	 Men in Black (a.k.a. MIB) (1997)
	 Dances with Wolves (1990)
Cluster #2
	 Independence Day (a.k.a. ID4) (1996)
	 Mission: Impossible (1996)
	 Star Wars: Episode I - The Phantom Menace (1999)
	 Batman Forever (1995)
	 Twister (1996)
	 Net, The (1995)
	 Sleepless in Seattle (1993)
	 Outbreak (1995)
	 Firm, The (1993)
	 While You Were Sleeping (1995)
Cluster #3
	 E.T. the Extra-Terrestrial (1982)
	 Ghost (1990)
	 Breakfast Club, The (1985)
	 Harry Potter and the Sorcerer's Stone (a.k.a. Harry Pott